In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

data = pd.read_csv('your path/spheroid_time_series.csv')


features = data.drop(columns=['ID', 'Condition']).fillna(0)
labels = data['Condition']


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


def evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        'Random Forest': RandomForestClassifier(),
        'SVM': SVC(),
        'Logistic Regression': LogisticRegression(max_iter=500),
        'KNN': KNeighborsClassifier(),
        'Naive Bayes': GaussianNB(),
        'Gradient Boosting': GradientBoostingClassifier()
    }

    results = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[name] = accuracy

    return results


results_all = {}


for n_components in range(2, 11):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)

    pca = PCA(n_components=n_components)
    X_pca_train = pca.fit_transform(X_scaled[X_train.index])
    X_pca_test = pca.transform(X_scaled[X_test.index])

    results = evaluate_models(X_train=X_pca_train, X_test=X_pca_test, y_train=y_train, y_test=y_test)
    results_all[n_components] = results


results_df = pd.DataFrame(results_all).T
results_df.index.name = 'Number of Features'
results_df = results_df.reset_index()

print(results_df)